In [1]:
import re

import pandas as pd
from more_itertools import unique_everseen
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from utils import get_hypernyms

## Load dataset with Ukrainian titles

In [2]:
filtered_uk = pd.read_csv("./data/titled_pwn.csv", sep=";").loc[
    lambda x: x["rel"] == "pwn31_to_uk_wiki"
]

## Filtering techniques 
1) Remove too general titles
2) Everything in brackets
3) Titles that are more than trigrams
4) Titles consisting of Latin letters

Привіт! Підготувала датасет гіпонім: тип: гіпероніми. 
Підхід був як у SemEval гіпонім і 5 його батьків гіперонімів. Застосувала наступні техніки для фільтрації:
1) Видалила занадто загальні заголовки ("Фізичне тіло", "Тверде тіло", "Матерія (фізика)", "Суще")
2) Все, що в дужках у заголовках: Колорадо (річка в Техасі) -> Колорадо
3) Назви, які є більшими за триграми
4) Назви, що складаються з латинських літер
5) Гіпоніми, у яких немає хоча б одного прямого гіпероніма

In [3]:
# Filter out rows with too general titles
filtered_uk = filtered_uk.loc[
    ~filtered_uk["title"].isin(
        ["Фізичне тіло", "Тверде тіло", "Матерія (фізика)", "Суще"]
    )
]

filtered_uk = filtered_uk.dropna().reset_index(drop=True)

pattern = r"\([^)]*\)"

filtered_uk["title"] = filtered_uk["title"].apply(
    lambda x: re.sub(pattern, "", x).strip()
)

# Temporary filter bigrams and trigrams
filtered_uk = filtered_uk[filtered_uk["title"].apply(lambda x: len(x.split())) < 2]

filtered_uk = filtered_uk[
    filtered_uk["title"].str.match(r".*[^\x00-\xFF]")
].reset_index(drop=True)
filtered_uk["title"] = filtered_uk["title"].str.lower()

filtered_uk

,id_from,id_to,rel,title
0,omw-en31-02920509-n,https://uk.wikipedia.org/wiki/%D0%9A%D0%BE%D1%...,pwn31_to_uk_wiki,корида
1,omw-en31-08723872-n,https://uk.wikipedia.org/wiki/%D0%90%D0%BB%D0%...,pwn31_to_uk_wiki,алжир
2,omw-en31-08725731-n,https://uk.wikipedia.org/wiki/%D0%90%D0%BD%D0%...,pwn31_to_uk_wiki,ангола
3,omw-en31-08789861-n,https://uk.wikipedia.org/wiki/%D0%93%D0%BE%D0%...,pwn31_to_uk_wiki,гоенлінден
4,omw-en31-04220420-n,https://uk.wikipedia.org/wiki/%D0%A1%D0%B5%D1%...,pwn31_to_uk_wiki,серп
...,...,...,...,...
13818,omw-en31-08170236-n,https://uk.wikipedia.org/wiki/%D0%9F%D0%B0%D0%...,pwn31_to_uk_wiki,пантеон
13819,omw-en31-04219349-n,https://uk.wikipedia.org/wiki/%D0%92%D0%BE%D0%...,pwn31_to_uk_wiki,волан
13820,omw-en31-08169784-n,https://uk.wikipedia.org/wiki/%D0%94%D1%83%D1%...,pwn31_to_uk_wiki,духовенство
13821,omw-en31-07730735-n,https://uk.wikipedia.org/wiki/%D0%91%D1%80%D0%...,pwn31_to_uk_wiki,броколі


In [53]:
def get_titles(lst):
    return (
        filtered_uk.set_index("id_from")
        .reindex(lst)["title"]
        .dropna()
        .drop_duplicates()
        .tolist()
    )


data = []

for idx, row in tqdm(filtered_uk.iterrows(), total=len(filtered_uk)):
    res = {}
    hypernyms, query_type = get_hypernyms(row["id_from"])
    if not get_titles(hypernyms):
        continue
    all_hypernyms = hypernyms[:]
    for _ in range(4):
        new_hypernyms = []
        for hypernym in all_hypernyms:
            indirect, _ = get_hypernyms(hypernym)
            new_hypernyms.extend(indirect)
        all_hypernyms.extend(new_hypernyms)
    all_hypernyms = list(unique_everseen(all_hypernyms))
    if all_hypernyms:
        hypernym_titles = get_titles(all_hypernyms)
        if hypernym_titles and (row["title"] not in hypernym_titles):
            res["query"] = row["title"]
            res["query_type"] = query_type
            res["hypernyms"] = hypernym_titles
            if res not in data:
                data.append(res)

  0%|          | 0/13823 [00:00<?, ?it/s]

In [58]:
len(data)

4890

## Split and write data in SemEval2018: Task 9 format 

In [57]:
SEED = 42
train_test_data, dev_data = train_test_split(
    data,
    train_size=4800,
    test_size=90,
    stratify=[d["query_type"] for d in data],
    random_state=SEED,
)

train_data, test_data = train_test_split(
    train_test_data,
    train_size=2400,
    test_size=2400,
    stratify=[d["query_type"] for d in train_test_data],
    random_state=SEED,
)


def write_data_to_file(data, filename):
    with open(f"{filename}.data.txt", "w") as f_data, open(
        f"{filename}.gold.txt", "w"
    ) as f_gold:
        for d in data:
            f_data.write(f"{d['query']}\t{d['query_type']}\n")
            f_gold.write("\t".join(d["hypernyms"]) + "\n")


write_data_to_file(train_data, "./data/discovery/1A.english.training")
write_data_to_file(test_data, "./data/discovery/1A.english.test")
write_data_to_file(dev_data, "./data/discovery/1A.english.trial")